**1. Cost changes for each product**

**There's a table called ProductCostHistory which contains the history of the cost of the product.**

**Using that table, get the total number of times the product cost has changed.**

**Sort the results by ProductID**

In [ ]:
SELECT 
    ProductID,
    TotalPriceChanges = COUNT(ModifiedDate)
FROM ProductCostHistory
GROUP BY ProductID


2. Customers with total orders placed

We want to see a list of all the customers that have made orders, and the total number of orders

the customer has made.

Sort by the total number of orders, in descending order

In [ ]:
SELECT 
    SO.CustomerID, 
    TotalOrders = COUNT(SalesOrderID)
FROM SalesOrderHeader SO
JOIN Customer C ON C.CustomerID = SO.CustomerID
GROUP BY SO.CustomerID
ORDER BY TotalOrders DESC 

3. Products with first and last order date

For each product that was ordered, show the first and last date that it was ordered.

In the previous problem I gave you the table name to use. For this problem, look at the list of

tables, and figure out which ones you need to use.

Sort the results by ProductID.

In [ ]:
SELECT 
    SD.ProductID,
    FirstOrder = MIN(CONVERT(date, OrderDate)),
    LastOrder = MAX(CONVERT(date,OrderDate))
FROM SalesOrderDetail SD 
JOIN SalesOrderHeader SH ON SD.SalesOrderID = SH.SalesOrderID
GROUP BY SD.ProductID
ORDER BY SD.ProductID

4. Products with first and last order date, including name

For each product that was ordered, show the first and last date that it was ordered. This time,

include the name of the product in the output, to make it easier to understand.

Sort the results by ProductID.

In [ ]:
SELECT 
    SD.ProductID,
    P.ProductName,
    FirstOrder = MIN(CONVERT(date, OrderDate)),
    LastOrder = MAX(CONVERT(date,OrderDate))
FROM SalesOrderDetail SD 
JOIN SalesOrderHeader SH ON SD.SalesOrderID = SH.SalesOrderID
JOIN Product P ON P.ProductID = SD.ProductID
GROUP BY SD.ProductID,P.ProductName
ORDER BY SD.ProductID

5. Product cost on a specific date

We'd like to get a list of the cost of products, as of a certain date, 2012-04-15. Use the

ProductCostHistory to get the results.

Sort the output by ProductID.

In [ ]:
SELECT 
    ProductID,
    StandardCost
FROM ProductCostHistory
WHERE '2012-04-15' BETWEEN StartDate AND EndDate
ORDER BY ProductID

6. Product cost on a specific date, part 2

It turns out that the answer to the above problem has a problem. Change the date to 2014-04-15.

What are your results?

If you use the SQL from the answer above, and just change the date, you won't get the results

you want.

Fix the SQL so it gives the correct results with the new date. Note that when the EndDate is null,

that means that price is applicable into the future.

In [ ]:
SELECT 
    ProductID,
    StandardCost
FROM ProductCostHistory
WHERE '2012-04-15' BETWEEN StartDate AND ISNULL(EndDate,getdate())
ORDER BY ProductID

7. Product List Price: how many price changes?

Show the months from the ProductListPriceHistory table, and the total number of changes made

in that month.

In [ ]:
SELECT 
    ProductListPriceMonth = FORMAT(StartDate, 'yyyy/MM'),
    TotolRows = COUNT(*)
FROM ProductListPriceHistory
GROUP BY FORMAT(StartDate, 'yyyy/MM')

In [ ]:
SELECT 
    ProductListPriceMonth = CONCAT(YEAR(StartDate), '/','0',MONTH(StartDate)),
    TotalRows = COUNT(*)
FROM ProductListPriceHistory
GROUP BY CONCAT(YEAR(StartDate), '/','0',MONTH(StartDate))

8. Product List Price: months with no price changes?

After reviewing the results of the previous query, it looks like price changes are made only in

one month of the year.

We want a query that makes this pattern very clear. Show all months (within the range of

StartDate values in ProductListPriceHistory). This includes the months during which no prices

were changed.

In [ ]:
SELECT
    CalendarMonth,
    TotalRows = COUNT(PH.StartDate)
FROM Calendar C
LEFT JOIN ProductListPriceHistory PH 
ON C.CalendarDate = PH.StartDate
WHERE 
    C.CalendarDate >=(SELECT MIN(StartDate)FROM ProductListPriceHistory)
AND 
    C.CalendarDate <=(SELECT MAX(StartDate)FROM ProductCostHistory)
GROUP BY CalendarMonth
ORDER BY CalendarMonth

9. Current list price of every product

What is the current list price of every product, using the ProductListPrice history?

Order by ProductID

In [ ]:
SELECT 
     ProductID,
     ListPrice
FROM ProductListPriceHistory
WHERE EndDate IS NULL 
ORDER BY ProductID

10. Products without a list price history

Show a list of all products that do not have any entries in the list price history table.

Sort the results by ProductID

In [ ]:
SELECT 
    p.ProductID,
    P.ProductName 
FROM ProductListPriceHistory PH
FULL JOIN Product P ON P.ProductID = PH.ProductID
WHERE PH.ListPrice IS NULL 
ORDER BY P.ProductID

In [ ]:
SELECT 
    ProductID,
    ProductName 
FROM Product
WHERE ProductID NOT IN (SELECT ProductID FROM ProductListPriceHistory)
ORDER BY ProductID


11. Product cost on a specific date, part 3

In the earlier problem “Product cost on a specific date, part 2”, this answer was given:

Select

ProductID

,StandardCost

From ProductCostHistory

Where

'2014-04-15' Between StartDate and IsNull(EndDate, getdate())

Order By ProductID

However, there are many ProductIDs that exist in the ProductCostHistory table that don’t show

up in this list.

In [ ]:
SELECT DISTINCT
    ProductID   
FROM ProductCostHistory
WHERE ProductID NOT IN (
SELECT 
    ProductID
FROM ProductCostHistory
WHERE '2014-04-15' BETWEEN StartDate AND ISNULL(EndDate, getdate())
)
ORDER BY ProductID

In [ ]:
SELECT 
    ProductID 
FROM ProductCostHistory
EXCEPT
SELECT 
    ProductID
FROM ProductCostHistory
WHERE '2014-04-15' BETWEEN StartDate AND ISNULL(EndDate, getdate())
ORDER BY ProductID

12. Products with multiple current list price records

There should only be one current price for each product in the ProductListPriceHistory table, but

unfortunately some products have multiple current records.

Find all these, and sort by ProductID

In [ ]:
WITH TotalCountofCurrentPrice AS (SELECT 
     ProductID,
     TotalCount = COUNT(ListPrice)
FROM ProductListPriceHistory
WHERE EndDate IS NULL
GROUP BY ProductID) 

SELECT 
    ProductID
FROM TotalCountofCurrentPrice TP 
WHERE TotalCount > 1
ORDER BY ProductID

In [ ]:
SELECT 
    ProductID
FROM ProductListPriceHistory
WHERE EndDate IS NULL 
GROUP BY ProductID
HAVING COUNT(*) > 1


13. Products with their first and last order date, including name

and subcategory

In the problem “Products with their first and last order date, including name", we looked only at

product that have been ordered.

It turns out that there are many products that have never been ordered.

This time, show all the products, and the first and last order date. Include the product

subcategory as well.

Sort by the ProductName field.

In [ ]:
SELECT 
     P.ProductID,
     P.ProductName,
     PS.ProductSubCategoryName,
     FirstOrder = CONVERT(date,MIN(OrderDate)),
     LastOrder = CONVERT(date,MAX(OrderDate))
FROM Product P 
FULL JOIN ProductSubcategory PS ON P.ProductSubcategoryID = PS.ProductSubcategoryID
FULL JOIN SalesOrderDetail SD ON P.ProductID = SD.ProductID
FULL JOIN SalesOrderHeader SH ON SD.SalesOrderID = SH.SalesOrderID
GROUP BY P.ProductID,P.ProductName,PS.ProductSubCategoryName
ORDER BY P.ProductName

14. Products with list price discrepancies

It's astonishing how much work with SQL and data is in finding and resolving discrepancies in

data. Some of the salespeople have told us that the current price in the price list history doesn't

seem to match the actual list price in the Product table.

Find all these discrepancies. Sort the results by ProductID.

In [ ]:
WITH LatestPrice AS (
    SELECT 
    ProductID,
    ListPrice
    FROM ProductListPriceHistory
    WHERE EndDate IS NULL 
)


SELECT 
    P.ProductID,
    P.ProductName,
    Prod_ListPrice = P.ListPrice,
    PriceHist_LastestList = LP.ListPrice,
    Diff = (P.ListPrice - LP.ListPrice)
FROM Product P 
JOIN LatestPrice LP ON P.ProductID = LP.ProductID
WHERE P.ListPrice != LP.ListPrice
ORDER BY ProductID

In [ ]:
SELECT 
    P.ProductID,
    P.ProductName,
    Prod_listPrice = P.ListPrice,
    ProdHis_lastestPrice = PH.ListPrice
FROM Product P 
JOIN ProductListPriceHistory PH ON P.ProductID = PH.ProductID
WHERE P.ListPrice !=PH.ListPrice
AND PH.EndDate IS NULL 
ORDER BY P.ProductID

15. Orders for products that were unavailable

It looks like some products were sold before or after they were supposed to be sold, based on the

SellStartDate and SellEndDate in the Product table. Show a list of these orders, with details.

Sort the results by ProductID, then OrderDate.

In [ ]:
SELECT 
    P.ProductID,
    OrderDate = CONVERT(date,SH.OrderDate),
    P.ProductName,
    Qty = SD.OrderQty,
    SellStartDate = CONVERT(date,P.SellStartDate),
    SellEndDate = CONVERT(date,P.SellEndDate)
FROM Product P
JOIN SalesOrderDetail SD ON P.ProductID = SD.ProductID
JOIN SalesOrderHeader SH ON SD.SalesOrderID = SH.SalesOrderID
WHERE SH.OrderDate > P.SellEndDate OR SH.OrderDate < P.SellStartDate
AND P.SellEndDate IS NOT NULL 
ORDER BY ProductID



In [ ]:
SELECT 
    P.ProductID,
    OrderDate = CONVERT(date,SH.OrderDate),
    P.ProductName,
    Qty = SD.OrderQty,
    SellStartDate = CONVERT(date,P.SellStartDate),
    SellEndDate = CONVERT(date,P.SellEndDate)
FROM Product P
JOIN SalesOrderDetail SD ON P.ProductID = SD.ProductID
JOIN SalesOrderHeader SH ON SD.SalesOrderID = SH.SalesOrderID
WHERE SH.OrderDate NOT BETWEEN P.SellStartDate AND ISNULL(P.SellEndDate,OrderDate)
ORDER BY ProductID,OrderDate



16. Orders for products that were unavailable: details

We'd like to get more details on when products that were supposed to be unavailable were

ordered.

Create a new column that shows whether the product was ordered before the sell start date, or

after the sell end date.

Sort the results by ProductID and OrderDate.

In [ ]:
SELECT 
    P.ProductID,
    OrderDate = CONVERT(date,SH.OrderDate),
    Qty = SD.OrderQty,
    SellStartDate = CONVERT(date,P.SellStartDate),
    SellEndDate = CONVERT(date,P.SellEndDate),
    ProblemTypes = CASE 
                   WHEN OrderDate > SellEndDate THEN 'Sold after end date'
                   WHEN OrderDate < SellStartDate THEN 'Sold before start date'
                END 
FROM Product P
JOIN SalesOrderDetail SD ON P.ProductID = SD.ProductID
JOIN SalesOrderHeader SH ON SD.SalesOrderID = SH.SalesOrderID
WHERE SH.OrderDate NOT BETWEEN P.SellStartDate AND ISNULL(P.SellEndDate,OrderDate)
ORDER BY ProductID,OrderDate

17. OrderDate with time component

How many OrderDate values in SalesOrderHeader have a time component to them?

Show the results as below.

In [ ]:
WITH TotalOrderWithTime AS (SELECT 
    TotalOrderWithTime = COUNT(*)
FROM SalesOrderHeader
WHERE OrderDate != CONVERT(date,OrderDate)) 

SELECT 
    TotalOrderWithTime = (SELECT * FROM TotalOrderWithTime),
    TotalOrders = COUNT(*),
    PercentOrdersWithTimes = (SELECT * FROM TotalOrderWithTime) * 1.00/ COUNT(*) 
 FROM SalesOrderHeader




18. Fix this SQL! Number 1

We want to show details about certain products (name, subcategory, first order date, last order

date), similar to what we did in a previous query.

This time, we only want to show the data for products that have Silver in the color field. You

know, by looking at the Product table directly, that there are many products that have that color.

A colleague sent you this query,

In [ ]:
Select
Product.ProductID
,ProductName
,ProductSubCategoryName
,FirstOrder = Convert(date, Min(OrderDate))
,LastOrder = Convert(date, Max(OrderDate))
,Product.Color
From Product
Left Join SalesOrderDetail Detail 
on Product.ProductID = Detail.ProductID AND Product.Color = 'Silver'
Left Join SalesOrderHeader Header
on Header.SalesOrderID = Detail .SalesOrderID
Left Join ProductSubCategory
on ProductSubCategory .ProductSubCategoryID = Product.ProductSubCategoryID
Group by 
Product.ProductID
,ProductName
,ProductSubCategoryName
,Product.Color
Order by LastOrder desc

19. Raw margin quartile for products

The product manager would like to show information for all products about the raw margin –

that is, the price minus the cost. Create a query that will show this information, as well as the raw

margin quartile.

For this problem, the quartile should be 1 if the raw margin of the product is in the top 25%, 2 if

the product is in the second 25%, etc.

Sort the rows by the product name.

In [ ]:
SELECT 
    ProductId, 
    ProductName,
    StandardCost,
    ListPrice,
    RawMargin = ListPrice - StandardCost,
    Quartile = NTILE(4)
               OVER(ORDER BY (ListPrice - StandardCost) DESC)
FROM Product
WHERE StandardCost !=0
AND ListPrice !=0
ORDER BY ProductName

20. Customers with purchases from multiple sales people

Show all the customers that have made purchases from multiple sales people.

Sort the results by the customer name (first name plus last name).

In [ ]:
SELECT
    SH.CustomerID, 
    CustomerName = CONCAT(FirstName,' ', LastName),
    TotalDifferentSalesPeople = COUNT(DISTINCT SH.SalesPersonEmployeeID)
FROM Customer C 
JOIN SalesOrderHeader SH ON C.CustomerID = SH.CustomerID
GROUP BY SH.CustomerID,CONCAT(FirstName,' ', LastName)
HAVING COUNT (DISTINCT SH.SalesPersonEmployeeID) > 1
ORDER BY CustomerName 

21. Fix this SQL! Number 2

A colleague has sent you the following SQL, which causes an error

In [ ]:
Select top 100
Customer.CustomerID
,CustomerName = FirstName + ' ' + LastName
,OrderDate
,SalesOrderHeader.SalesOrderID
,SalesOrderDetail.ProductID
,Product.ProductName
,LineTotal
From SalesOrderHeader
Join SalesOrderDetail
on SalesOrderHeader .SalesOrderID = SalesOrderDetail .SalesOrderID
Join Product
on Product.ProductID = SalesOrderDetail.ProductID
Join Customer
on Customer.CustomerID = SalesOrderHeader.CustomerID
Order by
CustomerID
,OrderDate

In [ ]:
SELECT 
    ProductName
FROM Product
GROUP BY ProductName
HAVING COUNT(*) > 1

23. Duplicate product: details

We'd like to get some details on the duplicate product issue. For each product that has duplicates,

show the product name and the specific ProductID that we believe to be the duplicate (the one

that's not the first ProductID for the product name).

In [ ]:
WITH DuplicatedRecordName AS (
    SELECT 
    ProductName
FROM Product
GROUP BY ProductName
HAVING COUNT(*) > 1
)

SELECT TOP 2
    PotentiaDuplicatedId = ProductID,
    DN.ProductName 
FROM Product P 
JOIN DuplicatedRecordName DN ON P.ProductName = DN.ProductName
ORDER BY PotentiaDuplicatedId DESC

In [ ]:
WITH Rowvalue AS (
    SELECT
        ProductId, 
        ProductName, 
        RowNumber= ROW_NUMBER()OVER(PARTITION BY ProductName ORDER BY ProductID)
    FROM Product) 

SELECT 
    PotentialDuplicateProductID = ProductId,
    ProductName
FROM Rowvalue
WHERE RowNumber != 1 

24. How many cost changes do products generally have?

We've worked on many problems based on the ProductCostHistory table. We know that the cost

for some products has changed more than for other products. Write a query that shows how

many cost changes that products have, in general.

For this query, you can ignore the fact that in ProductCostHistory, sometimes there's an

additional record for a product where the cost didn't actually change.

In [ ]:
WITH Pricechanges AS (SELECT 
    ProductID,
    TotalPriceChanges = COUNT(ModifiedDate)
FROM ProductCostHistory
GROUP BY ProductID) 

SELECT 
    TotalPriceChanges,
    TotalProducts = COUNT(ProductID)
FROM Pricechanges
GROUP BY TotalPriceChanges

25. Size and base ProductNumber for products

The ProductNumber field in the Product table comes from the vendor of the product. The size is

sometimes a part of this field.

We need to get the base ProductNumber (without the size), and then the size separately. Some

products do not have a size. For those products, the base ProductNumber will be the same as the

ProductNumber, and the size field will be null.

Limit the results to those ProductIDs that are greater than 533. Sort by ProductID.

In [ ]:
SELECT 
    ProductId, 
    ProductNumber,
    HyphoenLocation = CHARINDEX('-',ProductNumber), 
    BaseProductNumber = CASE   
                            WHEN CHARINDEX ('-',ProductNumber) = 0 THEN ProductNumber
                            ELSE SUBSTRING(ProductNumber, 1, CHARINDEX('-',ProductNumber)-1)
                        END,
    Size = CASE 
                WHEN CHARINDEX('-', ProductNumber) = 0 THEN NULL 
                ELSE SUBSTRING(ProductNumber, CharIndex('-',ProductNumber)+1, 2)
                END   
FROM Product P 
WHERE ProductID > 533
ORDER BY ProductID

In [ ]:
;With Main AS( 
    SELECT 
        ProductID,
        ProductName, 
        ProductNumber,
        HypenLocation = CHARINDEX('-', ProductNumber)
    FROM Product
    WHERE ProductID > 533
)

SELECT * , 
    BaseProductNumber = 
        CASE 
           WHEN HypenLocation = 0 THEN ProductNumber
           ELSE SUBSTRING(ProductNumber,1, HypenLocation -1)
           END,
    Size =  CASE
            WHEN HypenLocation = 0 THEN NULL 
            ELSE SUBSTRING(ProductNumber, HypenLocation+1, 2)
            END
FROM Main

26. Number of sizes for each base product number

Now we'd like to get all the base ProductNumbers, and the number of sizes that they have.<span style="color: rgb(0, 0, 0); font-family: &quot;Times New Roman&quot;; font-size: 12px;">Use the output of the previous problem to get the results. However, do not use the filter from the</span>

previous problem (ProductIDs that are greater than 533). Instead of that filter, select only those

products that are clothing (ProductCategory = 3).

Order by the base ProductNumber.

In [ ]:
;With Main AS( 
     SELECT 
        ProductID,
        ProductName, 
        ProductNumber,
        HypenLocation = CHARINDEX('-', ProductNumber)
    FROM Product P 
    WHERE ProductSubcategoryId IN 
    (SELECT ProductSubcategoryID FROM ProductSubcategory 
    WHERE ProductCategoryID = 3)
),
Sizes AS(SELECT * , 
    BaseProductNumber = 
        CASE 
           WHEN HypenLocation = 0 THEN ProductNumber
           ELSE SUBSTRING(ProductNumber,1, HypenLocation -1)
           END,
    Size =  CASE
            WHEN HypenLocation = 0 THEN NULL 
            ELSE SUBSTRING(ProductNumber, HypenLocation+1, 2)
            END
FROM Main) 

SELECT 
    BaseProductNumber, 
    TotalSize = COUNT(*)
FROM Sizes
GROUP BY BaseProductNumber


In [ ]:
;With Main AS( 
     SELECT 
        ProductID,
        ProductName, 
        ProductNumber,
        HypenLocation = CHARINDEX('-', ProductNumber)
    FROM Product P 
    JOIN ProductSubcategory PS ON P.ProductSubcategoryID = PS.ProductSubcategoryID
    WHERE ProductCategoryID = 3
),
Sizes AS(SELECT * , 
    BaseProductNumber = 
        CASE 
           WHEN HypenLocation = 0 THEN ProductNumber
           ELSE SUBSTRING(ProductNumber,1, HypenLocation -1)
           END,
    Size =  CASE
            WHEN HypenLocation = 0 THEN NULL 
            ELSE SUBSTRING(ProductNumber, HypenLocation+1, 2)
            END
FROM Main) 

SELECT 
    BaseProductNumber, 
    TotalSize = COUNT(*)
FROM Sizes
GROUP BY BaseProductNumber

27. How many cost changes has each product really had?

A sharp-eyed analyst has pointed out that the total number of product cost changes (from the

problem “Cost changes for each product” is not right. Why? Because sometimes, even when

there's a new record in the ProductCostHistory table, the cost is not actually different from the

previous record!

This eventually will require a fix to the database, to make sure that we do not allow a record like

this to be entered. This could be done as a table constraint, or a change to the code used to insert

the row.

However, for now, let's just get an accurate count of cost changes per product, where the cost has

actually changed. Also include the initial row for a product, even if there's only 1 record.

Sort the output by ProductID.

In [ ]:
WITH Main AS(
    SELECT 
        ProductID,
        StandardCost,
        PreviousStandCost = LAG(StandardCost,1)
                            OVER(PARTITION BY ProductID ORDER BY StartDate)
    FROM ProductCostHistory)

SELECT 
    P.ProductID, 
    P.ProductName, 
    TotalCostChanges = COUNT(*)
FROM Main M 
JOIN Product P ON M.ProductID = P.ProductID
WHERE M.PreviousStandCost != M.StandardCost
OR M.PreviousStandCost IS NULL 
GROUP BY P.ProductID, P.ProductName
ORDER BY P.ProductID

28. Which products had the largest increase in cost?

In [ ]:
WITH Main AS (
       SELECT 
        ProductID,
        StandardCost,
        PreviousStandardCost = LAG(StandardCost,1)
                               OVER(PARTITION BY ProductID ORDER BY StartDate)
    FROM ProductCostHistory

),
Cte AS (
    SELECT 
        ProductID,
        CostChangeDate = MAX(StartDate)
    FROM ProductCostHistory
    GROUP BY ProductID

)

SELECT 
    M.ProductID,
    C.CostChangeDate,
    M.StandardCost,
    M.PreviousStandardCost,
    PriceDifference = M.PreviousStandardCost - M.StandardCost
FROM Main M 
JOIN Cte C ON M.ProductID = C.ProductID
WHERE M.PreviousStandardCost - M.StandardCost IS NOT NULL 
ORDER BY PriceDifference DESC 


In [ ]:
WITH Main AS (
       SELECT 
        ProductID,
        StartDate, 
        StandardCost,
        PreviousStandardCost = LAG(StandardCost,1)
                               OVER(PARTITION BY ProductID ORDER BY StartDate)
    FROM ProductCostHistory) 

SELECT 
    ProductID,
    CostChangeDate = StartDate,
    StandardCost,
    PreviousStandardCost,
    PriceDifference = PreviousStandardCost -StandardCost
FROM Main 
WHERE PreviousStandardCost -StandardCost IS NOT NULL 
ORDER BY PriceDifference DESC


29. Fix this SQL! Number 3

In [ ]:
;with FraudSuspects as (
Select 
    *
From Customer
Where
CustomerID in (
29401
,11194
,16490
,22698
,26583
,12166
,16036
,25110
,18172
,11997
,26731
)
),
SampleCustomers AS (Select top 100 *
From Customer
Where
CustomerID not in (SELECT CustomerID FROM FraudSuspects)
ORDER BY NewID()) 

SELECT * FROM FraudSuspects
UNION ALL 
SELECT * FROM SampleCustomers

30. History table with start/end date overlap

In [ ]:
SELECT 
    C.CalendarDate,
    P.ProductID, 
    TotalTow = COUNT(*)
FROM Calendar C 
JOIN ProductListPriceHistory P
ON P.StartDate <= C.CalendarDate
AND P.EndDate >= CalendarDate
GROUP BY C.CalendarDate,P.ProductID
HAVING COUNT(*) > 1

In [ ]:
SELECT 
    C.CalendarDate,
    P.ProductID, 
    TotalTow = COUNT(*)
FROM Calendar C 
JOIN ProductListPriceHistory P
ON P.StartDate <= C.CalendarDate
AND ISNULL(P.EndDate, '2013-05-29') >= CalendarDate
GROUP BY C.CalendarDate,P.ProductID
HAVING COUNT(*) > 1
ORDER BY ProductID, C.CalendarDate

32. Running total of orders in last year

For the company dashboard we'd like to calculate the total number of orders, by month, as well

as the running total of orders.

Limit the rows to the last year of orders. Sort by calendar month.

In [ ]:
WITH FIlterORders AS(SELECT 
    CalendarMonth,
    TotalOrders = COUNT(SalesOrderID)
FROM Calendar C 
JOIN SalesOrderHeader S ON C.CalendarDate = CONVERT(date,S.OrderDate)
WHERE OrderDate >= DATEADD(YEAR,-1,(SELECT CONVERT(date,MAX(OrderDate))FROM SalesOrderHeader))
GROUP BY CalendarMonth) 

SELECT 
    CalendarMonth,
    TotalOrders,
    RunningTotal = SUM(TotalOrders) OVER(Order By CalendarMonth)
FROM FIlterORders
ORDER BY CalendarMonth
 

33. Total late orders by territory

Show the number of total orders, and the number of orders that are late.

For this problem, an order is late when the DueDate is before the ShipDate.

Group and sort the rows by Territory.

In [ ]:
WITH TotalOrders AS (SELECT 
    ST.TerritoryID,
    TerritoryName,
    CountryCode,
    TotalOrders = COUNT(*)
FROM SalesOrderHeader SO
JOIN SalesTerritory ST ON SO.TerritoryID = ST.TerritoryID
GROUP BY ST.TerritoryID, TerritoryName, CountryCode
) ,
Lateorders AS (SELECT 
    TerritoryID, 
    Totallateroders = COUNT(*)
FROM SalesOrderHeader 
WHERE DueDate <= ShipDate
GROUP BY TerritoryID
)

SELECT 
    T.TerritoryID,
    T.TerritoryName,
    T.CountryCode,
    T.TotalOrders,
    L.Totallateroders
FROM TotalOrders T
JOIN Lateorders L ON T.TerritoryID = L.TerritoryID
ORDER BY TerritoryID


In [ ]:
WITH Main AS ( 
    SELECT 
        SalesOrderID,
        TerritoryID,
        DueDate,
        ShipDate,
        OrderArrivedLate= 
            CASE 
                WHEN DueDate < ShipDate THEN 1
                ELSE 0
            END 
    FROM SalesOrderHeader
)

SELECT 
    ST.TerritoryID,
    ST.TerritoryName,
    ST.CountryCode,
    TotalOrders = Count(M.TerritoryID),--- Sum up all the individual records 
    TotalLateOrders = ISNULL(SUM(OrderArrivedLate),0)--- Where SUM OrderArrivedLate 
FROM SalesTerritory ST 
    LEFT JOIN Main M on ST.TerritoryID = M.TerritoryID
GROUP BY 
    ST.TerritoryID,
    ST.TerritoryName,
    CountryCode